In [1]:
import pycaret
from pycaret.clustering import *
import pandas as pd
import datetime

#get most recent clustering element, or specify by name
fn = 'efficiency_metric/2023-08-14 10:37:50.csv'
ec = pd.read_csv(fn)
ec.head()

,Unnamed: 0.1,Unnamed: 0,group,time,s_MP,change,type,length,sum_change,s_area,...,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,p_length,p_sum_change,p_area,Cluster,efficiency
0,0,0,1,1.660222e+12,29.99,0.317588,surge,1,0.317588,0.317588,...,30.00,7349599.5,1.229796e+09,716681.94,314676.40,6.0,-0.004943,-0.029656,Cluster 2,3.25%
1,1,1,3,1.660222e+12,29.85,0.000469,surge,1,0.000469,0.000469,...,29.90,7307990.0,1.229792e+09,715352.90,314642.84,1.0,0.000402,0.000402,Cluster 2,3.25%
2,2,2,5,1.660222e+12,29.87,0.001105,surge,2,0.001607,0.003214,...,29.92,7285245.0,1.229754e+09,714476.10,313576.00,4.0,-0.002773,-0.011090,Cluster 2,3.25%
3,3,3,7,1.660222e+12,29.89,0.001610,surge,1,0.001610,0.001610,...,29.88,7318707.0,1.229744e+09,715730.56,313546.60,2.0,-0.000200,-0.000400,Cluster 2,3.25%
4,4,4,9,1.660223e+12,29.90,0.001372,surge,1,0.001372,0.001372,...,29.93,7280582.0,1.229794e+09,714140.10,314889.38,7.0,-0.013670,-0.095690,Cluster 2,3.25%


In [2]:
ec.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'group', 'time', 's_MP', 'change', 'type',
       'length', 'sum_change', 's_area', 'surge_area', 'p_group', 'p_time',
       'p_change', 'p_type', 'p_MP', 'p_buyCap', 'p_askCap', 'p_totalBidVol',
       'p_totalAskVol', 'p_length', 'p_sum_change', 'p_area', 'Cluster',
       'efficiency'],
      dtype='object')

In [3]:
arr = ec['Cluster'].unique()
arr

array(['Cluster 2', 'Cluster 0', 'Cluster 1', 'Cluster 3'], dtype=object)

In [4]:
#scat
used = ec#[:4999]
def compareEfficiencyAndVolume():
    alt.data_transformers.disable_max_rows()

    eff = alt.Chart(used).mark_circle().encode(
        x='surge_area:Q',
        y='efficiency',
        color='Cluster:N', 
        size='surge_area:Q',
        tooltip=['surge_area', 'Cluster']
    ).properties(height=300, width=400,title='Efficiency, per Cluster')
    
    
    tbv = alt.Chart(used).transform_fold(
        ['Cluster 2', 'Cluster 0', 'Cluster 1', 'Cluster 3'],
        as_=['Cluster', 'efficiency']
    ).mark_bar(
        opacity=0.3,
        binSpacing=0
    ).encode(
        alt.X('p_totalBidVol:Q').bin(maxbins=100),
        alt.Y('count()').stack(True),
        alt.Color('Cluster:N'),
        tooltip=['Cluster', 'p_totalBidVol', 'count()']
    ).properties(height=300, width=400,title='Precursor Ask Volume vs Bid Volume')
    
    tav = alt.Chart(used).transform_fold(
        ['Cluster 2', 'Cluster 0', 'Cluster 1', 'Cluster 3'],
        as_=['Cluster', 'efficiency']
    ).mark_bar(
        opacity=0.7,
        binSpacing=0
    ).encode(
        alt.X('p_totalAskVol:Q').bin(maxbins=100),
        alt.Y('count()').stack(True),
        alt.Color('Cluster:N'),
        tooltip=['Cluster', 'p_totalAskVol', 'count()']
    ).properties(height=300, width=400)
    return eff | tbv + tav

In [5]:
#scat
used = ec#[:4999]
def compareEfficiencyAndCaps():
    alt.data_transformers.disable_max_rows()

    eff = alt.Chart(used).mark_circle().encode(
        x='surge_area:Q',
        y='efficiency',
        color='Cluster:N', 
        size='surge_area:Q',
        tooltip=['surge_area', 'Cluster']
    ).properties(height=300, width=400,title='Efficiency, per Cluster')
    
    
    tbv = alt.Chart(used).transform_fold(
        ['Cluster 2', 'Cluster 0', 'Cluster 1', 'Cluster 3'],
        as_=['Cluster', 'efficiency']
    ).mark_bar(
        opacity=0.3,
        binSpacing=0
    ).encode(
        alt.X('p_buyCap:Q').bin(maxbins=100),
        alt.Y('count()').stack(True),
        alt.Color('Cluster:N'),
        tooltip=['Cluster', 'p_buyCap', 'count()']
    ).properties(height=300, width=400,title='Precursor Ask Capitaliztion vs Bid')
    
    tav = alt.Chart(used).transform_fold(
        ['Cluster 2', 'Cluster 0', 'Cluster 1', 'Cluster 3'],
        as_=['Cluster', 'efficiency']
    ).mark_bar(
        opacity=0.7,
        binSpacing=0
    ).encode(
        alt.X('p_askCap:Q').bin(maxbins=100),
        alt.Y('count()').stack(True),
        alt.Color('Cluster:N'),
        tooltip=['Cluster', 'p_askCap', 'count()']
    ).properties(height=300, width=400)
    return eff | tbv + tav

In [6]:
def buildMultiline():
    data = ec[['time','p_buyCap', 'p_askCap']].melt('time', var_name='variable', value_name='value')
    
    chart = alt.Chart(data).mark_line().encode(
        x='time:T',
        y=alt.Y('value:Q', scale=alt.Scale(zero=False)),
        color='variable:N'
    ).properties(
        width=600,
        height=400,
        title='Capitalization of Buy Orders vs Sell'
    )
    return chart

In [7]:
compareEfficiencyAndVolume()


NameError: name 'alt' is not defined

In [ ]:
## bid and ask volume time series vs mp
compareEfficiencyAndCaps()

In [ ]:
buildMultiline()